In [27]:
import pandas as pd
import urllib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import pickle
import preprocessor as p
import numpy as np

In [28]:
!dir

 Volume in drive C is OS
 Volume Serial Number is 54F6-FEB0

 Directory of C:\Users\Maheshwaran\Downloads\Detecting-Cyberbullying-Across-SMPs-master\Detecting-Cyberbullying-Across-SMPs-master\data_analysis

27-10-2021  11:46    <DIR>          .
27-10-2021  11:46    <DIR>          ..
29-10-2019  14:20    <DIR>          .ipynb_checkpoints
27-10-2021  11:34    <DIR>          data
27-10-2021  09:56            21,809 DataAnalysisFormspring.ipynb
27-10-2021  11:46             9,464 DataAnalysisTwitter.ipynb
27-10-2021  11:07            19,496 DataAnalysisWiki.ipynb
29-10-2019  14:20             3,289 swear_words.txt
27-10-2021  10:56         2,333,589 twitter_data.pkl
               5 File(s)      2,387,647 bytes
               4 Dir(s)  264,115,597,312 bytes free


In [29]:
import string

def load_data():
    filename = "data/twitter_data.pkl"
    print("Loading data from file: " + filename)
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = [] 
    ids = []
    for i in range(len(data)):
        text = "".join(l for l in data[i]['text'] if l not in string.punctuation)
        x_text.append((data[i]['text']).encode('utf-8'))
        labels.append(data[i]['label'])
    return x_text,labels

In [30]:
x_text, labels = load_data()
dict1 = {'racism':1,'sexism':1,'none':0}  # Both are 1 because both are cases of bullying
labels = np.array([dict1[b] for b in labels])

Loading data from file: data/twitter_data.pkl


In [31]:
comments = pd.DataFrame({'comment': x_text, 'attack': labels})

In [32]:
comments.head()

,comment,attack
0,b'rt @colonelkickhead: another bloody instant ...,0
1,b'@azzamalirhabi @jihadia8 this video of the p...,0
2,"b""oh really ? no more instant restaurants ? ...",0
3,b'rt @benfrancisallen: it has not been a good ...,0
4,b'rt @notofeminism: i don\xe2\x80\x99t need fe...,0


In [25]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Maheshwaran\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
swear_words = "swear_words.txt"
word_list = []
with open(swear_words) as f:
    word_list = f.read().splitlines()

In [34]:
from textblob import TextBlob

def tokens(post):
    return TextBlob(post.decode('utf8')).words
    
def get_bad_word_count(post):
    x = tokens(post)
    count = 0
    bad_words = []
    for word in x:
        if word in word_list:
            count+=1
            bad_words.append(word)
    return count, str(bad_words)

comments['bad_word_count'],comments['bad_word_list']= zip(*comments['comment'].map(get_bad_word_count)) 

In [35]:
def data_analysis(comments):
    total = len(comments)
    bully = len(comments[comments['attack']==1])
    swear = len(comments[comments['bad_word_count']>0])
    swear_bully = len(comments[(comments['bad_word_count']>0) & (comments['attack']==1) ])

    P_B = float(bully)/total
    P_S = float(swear)/total
    P_B_S = float(swear_bully)/swear
    P_S_B = float(swear_bully)/bully
    
    print("P(B): " + str(P_B))
    print("P(S): " + str(P_S))
    print("P(B|S): " + str(P_B_S))
    print("P(S|B): " + str(P_S_B))
    

In [36]:
data_analysis(comments)

P(B): 0.31410814170292106
P(S): 0.13499067743940335
P(B|S): 0.42955801104972374
P(S|B): 0.1846062524732885
